In [ ]:
#load table from SPXintraday excel file from Downloads

import pandas as pd
import os
import datetime
import numpy as np
import matplotlib.pyplot as pyplot

#load data
df = pd.read_excel('SPXintraday.xlsx')

#calculate returns
df['Return'] = df['Last'].pct_change()
df = df.drop(df[df['BarNo'] == 1.0].index)    
df = df.drop(df[df['BarNo'] > 79].index)

# plot return as y and BarNo as x
pyplot.plot(df['BarNo'], df['Return'], 'o')

average_return = df['Return'].mean()
df['Return'] = df['Return'] - average_return

# plot deseasoned return as y and BarNo as x
pyplot.plot(df['BarNo'], df['Return'], 'o')

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [ ]:
print(len(df[df['Return']!=0]))
print(len(df[df['Return']==0]))

print(df[df['Return']==0])

In [ ]:
x = df['BarNo'].to_numpy()
y = np.log(np.square(df['Return'].to_numpy()))

poly = PolynomialFeatures(degree=2)
poly_features = poly.fit_transform(x.reshape(-1, 1))

poly_reg_model = LinearRegression()
poly_reg_model.fit(poly_features, y)

poly_features_with_zeros = poly.fit_transform(df['BarNo'].to_numpy().reshape(-1, 1))
y_predicted = np.sqrt(np.exp(poly_reg_model.predict(poly_features_with_zeros)))
x = df['BarNo'].to_numpy()

print(poly_reg_model.coef_)
pyplot.plot(x, y_predicted, 'o')

In [ ]:
error = (df['Return'] - y_predicted).to_numpy(dtype='float64')
df['Error'] = error

pyplot.plot(range(len(x)), error)

In [ ]:
from scipy.optimize import minimize

def con(t):
    return 1 - t[1] - t[2]

def con2(t):
    return t[0]

def con3(t):
    return t[2]

def con4(t):
    return t[1]

cons = [{'type':'ineq', 'fun': con},
        {'type':'ineq', 'fun': con2},
        {'type':'ineq', 'fun': con3},
        {'type':'ineq', 'fun': con4}]

def neg_log_likelihood(params):
    # print(params)
    var = np.zeros(len(error),dtype='float64')
    var[0] = error.var()
    sum = 0
    for i in range(1,len(error)):
        var[i] = abs(params[0] + params[1]*(error[i-1]**2) + params[2]*var[i-1])

        sum += np.log(var[i]) + (error[i]**2)/var[i]

    return sum

params = [7e-9,0.11,0.88]
res = minimize(neg_log_likelihood, params, constraints=cons, method='COBYQA')
print(res)
print(res.x)


In [ ]:
params = res.x
# params = [7e-9,0.11,0.88]
# params = [ 8.335e-07,  4.975e-02, -2.310e-01]
# params = [7.80399236e-07, 5.06476993e-02, 2.71058634e-01]
# params = [2.46250142e-07, 5.14189152e-02, 2.67593138e-01]

var = [error.var()]
for i in range(1,len(error)):
    var.append(abs(params[0] + params[1]*error[i-1]**2 + params[2]*var[i-1]))
    # if(var[i] <= 0):
    #     var[i] = 0.00000001
    # if(var[i] == np.inf):
    #     var[i] = 1e20
    
pyplot.plot(range(len(error)), var)


In [ ]:
df['Var'] = np.sqrt(var)
filtered = df[abs(df['Error']) > 2 * df['Var']]
# print(filtered['BarNo'].value_counßts().sort_index())
bars = np.zeros(14)
total = np.zeros(14)
for row in filtered.iterrows():
    bars[int(row[1]['BarNo'])//6] += 1
for row in df.iterrows():
    total[int(row[1]['BarNo'])//6] += 1    
x = [9*60+30*(i+1) for i in range(13)]
x = [f"{t//60}:{t%60//10}0" for t in x]
pyplot.figure(figsize=(10,6))
bars = bars/total
pyplot.bar(x,bars[:13])

In [ ]:

pyplot.plot(range(len(var))[:237], (y_predicted*var)[:237])
pyplot.axvline(x=80, color='r')
pyplot.axvline(x=160, color='r')